# Exercise 1
Obtain the general formulation for the Google Adwords problem (described within slides 23-26 of the Linear Programming topic).

**Data:**

$a_i = $ maximum budget for company

$b_j = $ maximum of expected request for query

$c_{ij} = $ price per add/click

**Variables:**

$x_{ij} = $ Number of adds assigned per query and company

$$\max_{x_{ij}} \quad \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij}$$

$$s.t. \quad \sum_{i=1}^n x_{ij} \leq b_j \quad \forall j$$
$$\sum_{j=1}^m c_{ij} x_{ij} \leq a_i \quad \forall i$$
$$x_{ij} \geq 0 \quad ij$$

# Exercise 2
Obtain its equivalent standard form.

$$ \max_x \quad z = c^{\top}x $$
$$s.t. \quad Ax = b $$
$$x \geq =0 $$

where $x \in {\rm I\!R}^n, c \in {\rm I\!R}^n, A \in {\rm I\!R}^{m\times n}, and b \in {\rm I\!R}^m$


# Exercise 3
Implement the model derived in 2) in Pyomo and solve it for the case of 10 companies and 10 queries (make up the data to be reasonable). Compute the sensitivities associated to each constraint. Report the results.

In [13]:
%%writefile 1st_assignement3.py

#structure of the code from https://github.com/Pyomo/PyomoGettingStarted/blob/master/01_PyomoOverview.ipynb

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.Ia = RangeSet(1, model.m+model.n)
model.Ja = RangeSet(1, model.m*model.n)

model.a = Param(model.Ia, model.Ja)
model.ba = Param(model.Ia)
model.ca = Param(model.Ja)

# the next line declares a variable indexed by the set J
model.x = Var(model.Ja, domain=NonNegativeReals)

def obj_expression(model):
    return summation(model.ca, model.x)

#std form - minimize
model.OBJ = Objective(rule=obj_expression)

def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.Ja) == model.ba[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.Ia, rule=ax_constraint_rule)


Overwriting 1st_assignement3.py


In [14]:
%%writefile 1st_assignement3.dat


param n :=3; #Companies
param m :=3; #Queries
    
param ca := #revenue - as negative for the std form
    1 -1
    2 -0.75
    3 -5
    4 -0.5
    5 -0.5
    6 -2
    7 -0.5 
    8 -3
    9 -1
    ;
       
param ba:=
    1 200
    2 150
    3 180
    4 150
    5 90
    6 80
    ;

param a: 1 2 3 4 5 6 7 8 9 :=
1 1 0.75 5 0 0 0 0 0 0
2 0 0 0 0.5 0.5 2 0 0 0
3 0 0 0 0 0 0 0.5 3 1
4 1 0 0 1 0 0 1 0 0
5 0 1 0 0 1 0 0 1 0
6 0 0 1 0 0 1 0 0 1
;

Overwriting 1st_assignement3.dat


In [15]:
!pyomo solve 1st_assignement3.py 1st_assignement3.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.03] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: -530.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=9, Index=Ja
        Key : Lower : Value         : Upper : Fixed : Stale : Domain
          1 :     0 :           0.0 :  None : False : False : NonNegativeReals
          2 :     0 : 35.1851851852 :  None : False : False : NonNegativeReals
          3 :     0 : 34.7222222222 :  None : False : False : NonNegativeReals
          4 :     0 : 118.888888889 :  None : False : False : NonNegativeReals
          5 :     0 :           0.0 :  None : False : False : NonNegativeReals
          6 :     0 : 45.2777777778 :  None : False : False : NonNegativeReals
          7 :     0 : 31.1111111111 :  None : False : Fals

In [16]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -530.0
  Upper bound: -530.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0200002193451
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------

In [37]:
%%writefile 1st_assignement3_dual.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.Ia = RangeSet(1, model.m+model.n)
model.Ja = RangeSet(1, model.m*model.n)

model.a = Param(model.Ia, model.Ja)
model.ba = Param(model.Ia)
model.ca = Param(model.Ja)

# the next line declares a variable indexed by the set J
model.y = Var(model.Ia, domain=Reals)

#maximize
def obj_expression(model):
    return summation(model.ba, model.y)

model.OBJ = Objective(rule=obj_expression)

#we need to do At*y, but changing the indexes to 
#access the matrix does the trick
def ax_constraint_rule(model, j):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.y[i] for i in model.Ia) >= model.ca[j]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.Ja, rule=ax_constraint_rule)

Overwriting 1st_assignement3_dual.py


In [38]:
!pyomo solve 1st_assignement3_dual.py 1st_assignement3.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.05] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: -530.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    y : Size=6, Index=Ia
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  -1.0 :  None : False : False :  Reals
          2 :  None :  -1.0 :  None : False : False :  Reals
          3 :  None :  -1.0 :  None : False : False :  Reals
          4 :  None :   0.0 :  None : False : False :  Reals
          5 :  None :   0.0 :  None : False : False :  Reals
          6 :  None :   0.0 :  None : False : False :  Reals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -530.0

  Constraints:
    AxbConstraint : Size=9
        Key :

# Exercise 4
Given a linear programming problem in standard form:
$$\min_{x} \quad z_P = c^{\top} x $$ 
$$s.t. \quad Ax = b $$
$$x \geq 0 $$
we can define its dual problem as 
$$\max_{y} \quad z_D = b^{\top} y $$
$$s.t. \quad A^{\top}y\geq c $$
where $y$ is called teh dual variable vector.

Considering this, formulate the dual problem associated to the model derived in 2) (check slide 60 of the Linear Programming topic for an example of this transformation).

# Exercise 5
Implement the dual model derived in 4) in Pyomo and solve it for the same data in 3). Report the results.

# Exercise 6
The Strong Duality Theorem states that:

If $x^*$ is the optimal solution of the primal minimization LP problem, and $y^*$ is the dual optimal solution of the corresponding dual maximization LP problem, then

$$z^*_D = b^{\top} y^* = c^{\top} x^* = z^*_P $$

Comparing the solutions in 3) and 5), check if the Strong Duality Theorem holds. What is the relationship between the sensitivities computed in 3) and the optimal value of the dual variables obtained in 5)?

# Exercise 7
Imagine now that google is able to display simultaneously n company ads for each requested query (instead of only one). Moreover, assume that the specific order in which these ads are displayed is important. Indicate how the model in 1) would need to be modified to account for these facts. Also, indicate if any additional problem data will be necessary for this new setting.

# IMPORTANT:

Upload the formulations 1), 2), 4) and 7) as well as the answers to the different questions in a pdf file (generated with MSWord, latex or similar) and the codes for 3) and 5) as separated .py and .dat files.

This assignment can be done individually or in groups (up to 4 components). Groups must be the same for all the assignments. Files need to be uploaded by only one member of the group. Do not forget to indicate in the pdf file all the group’s components.